In [144]:
import os
import pandas as pd
from itertools import islice
import re

### 1 - Load and parse helper files for loading raw OPD data file

In [3]:
data_path = os.path.join('..', 'data', 'raw')

#helpers = []
#for file in os.listdir(data_path):
#    if file.endswith('.sps'):
#        helpers.append(pd.read_csv(os.path.join(data_path, file)))

In [154]:
with open(os.path.join(data_path, 'opd2010.sps'), 'r') as f:
    # Keep the trunk after 'DATA LIST...' and before the first '.'
    for i, line in enumerate(f):
        if line.startswith('DATA LIST'):
            start = i + 1
        if line.startswith('.'):
            end = i
            break

    #helper = [line.strip() for line in helper[start:end]]
    f.seek(0)
    helper = list(islice(f, start, end))
    helper = [line.strip() for line in helper]

# Convert the helper to a DataFrame, split each line by whitespace
helper = pd.DataFrame([line.replace(' (A)', '').split(maxsplit=1) for line in helper]).rename(columns={0:'name'})

# Remove the rows that ends with digits followed by 'D'
helper = helper[~helper['name'].str.contains(r'\dD$')]

In [155]:
helper['width'] = helper[1].apply(lambda x: abs(eval(x)) + 1 if ('-' in x) else 1)

In [156]:
opd2010 = pd.read_fwf(os.path.join(data_path, 'opd2010'), widths=helper['width'], names=helper['name'], header=None)

In [161]:
opd2010.iloc[:, 40:60]

,DEPRN,DIABETES,HYPLIPID,HTN,IHD,OBESITY,OSTPRSIS,NOCHRON,TOTCHRON,HTIN,WTLB,BMI,TEMPF,BPSYS,BPDIAS,SERVICES,BREAST,FOOT,PELVIC,RECTAL
0,0,0,0,0,0,0,0,1,0,33,26,-7.00,-9,-9,-9,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0,46,47,15.61,-9,-9,-9,1,0,0,0,0
2,0,0,0,0,0,0,0,1,0,25,19,-7.00,-9,-9,-9,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,62,145,26.52,994,116,68,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,34,28,17.03,-9,-9,-9,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34713,0,0,0,0,0,1,0,0,1,-9,240,-7.00,-9,118,70,1,0,0,0,0
34714,0,1,0,1,0,1,0,0,4,-9,313,-9.00,981,130,74,1,0,0,0,0
34715,0,0,0,0,0,0,0,1,0,-9,135,-7.00,-9,104,64,1,0,0,0,0
34716,0,0,0,0,0,0,0,1,0,-9,-9,-7.00,-9,-9,-9,0,0,0,0,0


In [168]:
opd2010.PROC1.value_counts()

PROC1
-9      22320
8939     1793
897-     1678
8917      473
9509      358
        ...  
8551        1
8789        1
5569        1
0018        1
0821        1
Name: count, Length: 648, dtype: int64